In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import sigmoid
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

import numpy as np

In [2]:
class EmojiDataset(Dataset):
    def __init__(self, rgb_images, labels, transform=None):
        self.rgb_images = rgb_images  # (num, 224, 224, 3)
        self.labels = labels  # (num, 11)
        self.transform = transform

    def __len__(self):
        return len(self.rgb_images)

    def __getitem__(self, idx):
        # Convert to PyTorch tensors
        rgb = torch.tensor(self.rgb_images[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        # Permute from (H, W, C) → (C, H, W)
        rgb = rgb.permute(2, 0, 1)  # (3, 224, 224)
        # Normalize image pixels to [0,1]
        rgb /= 255.0

        if self.transform:
            rgb = self.transform(rgb)
        
        return rgb, label

In [3]:
# Load images and labels
rgb_train= np.load("img_train_plaintext_top11_rgb.npy")  # Shape: (num_train, 224, 224, 3)
labels_train = np.load("img_train_plaintext_top11_labels.npy")  # Shape: (num_train, 11) - Multi-label one-hot

rgb_valid= np.load("img_valid_plaintext_top11_rgb.npy")  # Shape: (num_valid, 224, 224, 3)
labels_valid = np.load("img_valid_plaintext_top11_labels.npy")  # Shape: (num_valid, 11)

rgb_test = np.load("img_test_plaintext_top11_rgb.npy")  # Shape: (num_valid, 224, 224, 3)
labels_test = np.load("img_test_plaintext_top11_labels.npy")  # Shape: (num_valid, 11)

print(rgb_train.shape, rgb_train.dtype, type(rgb_train[0]), rgb_train[0].shape)
print(labels_train.shape, labels_train.dtype, type(labels_train[0]), labels_train[0], labels_train[0][4])

(109176, 224, 224, 3) uint8 <class 'numpy.ndarray'> (224, 224, 3)
(109176, 11) int64 <class 'numpy.ndarray'> [0 0 0 0 1 0 0 0 0 0 0] 1


In [4]:
# Define transformations (optional)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),
])

# Create Dataset instances
train_dataset = EmojiDataset(rgb_train, labels_train, transform=transform)
valid_dataset = EmojiDataset(rgb_valid, labels_valid, transform=None)
test_dataset = EmojiDataset(rgb_test, labels_test, transform=None)

# Create DataLoaders
batch_size = 32  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
rgb, label = train_dataset[0]

print("Single image shape:", rgb.shape)  # Expected: (3, 224, 224)
print("Single label shape:", label.shape)  # Expected: (11,)

Single image shape: torch.Size([3, 224, 224])
Single label shape: torch.Size([11])


In [6]:
for rgb, labels in train_loader:
    print("Batch image shape:", rgb.shape)  # Expected: (32, 3, 224, 224)
    print("Batch label shape:", labels.shape)  # Expected: (32, 11)
    break

Batch image shape: torch.Size([32, 3, 224, 224])
Batch label shape: torch.Size([32, 11])


In [7]:
# Load Pretrained ResNet18
model = models.efficientnet_b0(weights="DEFAULT")

num_classes = 11  # Top 11

# Modify the last layer
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)
print(model)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [8]:
# Define loss function & optimizer
criterion = nn.BCEWithLogitsLoss()  # Multi-label loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 20  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels) 
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # After each epoch, validate on the validation set
    model.eval()  # Set model to evaluation mode
    valid_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device) 

            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)} , Valid Loss: {valid_loss/len(valid_loader)}")
    

Epoch 1, Train Loss: 0.27397516364578905 , Valid Loss: 0.2735544336299417
Epoch 2, Train Loss: 0.26881385474942915 , Valid Loss: 0.27030922736598073
Epoch 3, Train Loss: 0.26603497460414627 , Valid Loss: 0.2688964382208214
Epoch 4, Train Loss: 0.2637916405107673 , Valid Loss: 0.2672668199532877
Epoch 5, Train Loss: 0.26197573290166093 , Valid Loss: 0.2709708491724635
Epoch 6, Train Loss: 0.2604200209142664 , Valid Loss: 0.2648071761799868
Epoch 7, Train Loss: 0.25874731027175085 , Valid Loss: 0.26397838069017604
Epoch 8, Train Loss: 0.2572460505687757 , Valid Loss: 0.2597399680781617
Epoch 9, Train Loss: 0.2554724046355605 , Valid Loss: 0.26367700990861054
Epoch 10, Train Loss: 0.25367521347012917 , Valid Loss: 0.26112260259490794
Epoch 11, Train Loss: 0.25175308934898755 , Valid Loss: 0.2621496686780894
Epoch 12, Train Loss: 0.2496798516699885 , Valid Loss: 0.2604286284752624
Epoch 13, Train Loss: 0.24752932062846414 , Valid Loss: 0.26074811457483854
Epoch 14, Train Loss: 0.2448780533

In [9]:
model.eval()

for threshold in (0.3, 0.5, 0.7):
    print(f"Threshold: {threshold}")
    
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            # Convert logits to probabilities
            probs = sigmoid(outputs)  
            # Convert logits to binary, 
            # Threshold at 0.5 (adjustable) to get binary labels
            predicted = (probs > threshold).float()
            
            # Store predictions and actual labels
            all_preds.append(predicted.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
            correct += (predicted == labels).sum().item()
            total += labels.numel()  # Count total number of elements

    # Convert list of numpy arrays into a single numpy array
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    # Compute confusion matrix, precision, and recall for each label (emoji)
    accuracy = 100 * correct / total
    precision = 100 * precision_score(all_labels, all_preds, average="macro")  # Use 'macro' for multi-label
    recall = 100 * recall_score(all_labels, all_preds, average="macro")
    f1 = 100 * f1_score(all_labels, all_preds, average="macro")

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Precision: {precision:.2f}%")
    print(f"Test Recall: {recall:.2f}%")
    print(f"Test F1: {f1:.2f}% \n")

Threshold: 0.3
Test Accuracy: 88.36%
Test Precision: 38.48%
Test Recall: 18.94%
Test F1: 20.37% 

Threshold: 0.5
Test Accuracy: 90.30%
Test Precision: 55.53%
Test Recall: 9.95%
Test F1: 14.78% 

Threshold: 0.7
Test Accuracy: 90.55%
Test Precision: 63.73%
Test Recall: 5.17%
Test F1: 9.15% 



c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
